In [7]:
import sys
sys.path.append('../../../../')

In [8]:
from dataclasses import dataclass
import pprint
import itertools
import requests
import pandas as pd
import typing

from utils.subgraph_utils.constants import SUBGRAPH_API

In [9]:
def get_pool_data(api):

    query = """
    {
        platforms {
            pools {
                coins
                coinDecimals
                address
            }
        }
    }
    """
    r = requests.post(api, json={"query": query})
    data = dict(r.json())
    pool_data = data["data"]["platforms"][0]["pools"]

    return pool_data

In [10]:
network_name = 'Mainnet'

In [11]:
data = get_pool_data(SUBGRAPH_API[network_name])
data[:2]

[{'coins': ['0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
   '0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'],
  'coinDecimals': ['18', '18'],
  'address': '0x0043fcb34e7470130fde28198571dee092c70bd7'},
 {'coins': ['0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415',
   '0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050'],
  'coinDecimals': ['18', '18'],
  'address': '0x01fe650ef2f8e2982295489ae6adc1413bf6011f'}]

In [52]:
@dataclass
class Coin:
    """A dataclass cacheing some coin info and a few basic methods."""
    address: str
    network: str
    decimals: int

    def convert_to_float(self, input_amount: int) -> float:
        return input_amount / 10 ** self.decimals


@dataclass
class Hop:
    """A dataclass containing hops between two assets"""
    address: str
    network: str
    coin_a: Coin
    coin_b: Coin


@dataclass
class Route:
    """A dataclass containing multi hops between coin a and coin b"""
    n_hops: int
    hops: typing.List[Hop]
    coin_a: str
    coin_b: str

In [84]:
one_hops = []
all_coins = []
all_hops = []
for pool in data:
    pool_address = pool['address']
    coins = []    
    for idx, coin in enumerate(pool['coins']):
        coin_dataclass = Coin(address=coin, network=network_name, decimals=int(pool['coinDecimals'][idx]))
        coins.append(coin_dataclass)
        all_coins.append(coin)
    
    coin_permutations = list(itertools.permutations(coins))
    for coin_permutation in coin_permutations:

        hop = Hop(address=pool_address, network=network_name, coin_a=coin_permutation[0], coin_b=coin_permutation[1])
        all_hops.append(hop)
        multi_hop = Route(n_hops=1, hops=[hop], coin_a=hop.coin_a.address, coin_b=hop.coin_b.address)
        one_hops.append(multi_hop)

pprint.pprint(coins[0])
pprint.pprint(one_hops[0])

Coin(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff',
     network='Mainnet',
     decimals=18)
Route(n_hops=1,
      hops=[Hop(address='0x0043fcb34e7470130fde28198571dee092c70bd7',
                network='Mainnet',
                coin_a=Coin(address='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                            network='Mainnet',
                            decimals=18),
                coin_b=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                            network='Mainnet',
                            decimals=18))],
      coin_a='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
      coin_b='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490')


In [87]:
all_coins = list(set(all_coins))

In [55]:
two_hops = []
for i in one_hops:

    coin_a = i.coin_a

    for j in one_hops:

        # we only want connecting pools
        if not i.coin_b == j.coin_a:
            continue

        # we dont want to go back:
        if j.coin_b == coin_a:
            continue

        coin_b = j.coin_b
        multi_hop = Route(n_hops=2, hops=[i, j], coin_a=coin_a, coin_b=coin_b)
        two_hops.append(multi_hop)

pprint.pprint(two_hops[0])
print(f"Number of 2 hop routes: {len(two_hops)}")

Route(n_hops=2,
      hops=[Route(n_hops=1,
                  hops=[Hop(address='0x0043fcb34e7470130fde28198571dee092c70bd7',
                            network='Mainnet',
                            coin_a=Coin(address='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                                        network='Mainnet',
                                        decimals=18),
                            coin_b=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                                        network='Mainnet',
                                        decimals=18))],
                  coin_a='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                  coin_b='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'),
            Route(n_hops=1,
                  hops=[Hop(address='0x0212133321479b183637e52942564162bcc37c1d',
                            network='Mainnet',
                            coin_a=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                 

In [56]:
three_hops = []
for i in one_hops:  # first hop

    coin_a = i.coin_a

    for j in one_hops:  # second hop

        # we only want connecting pools
        if not i.coin_b == j.coin_a:
            continue

        # we don't want to go back:
        if j.coin_b == coin_a:
            continue

        for k in one_hops:  # third hop

            # we only want connecting pools
            if j.coin_b != k.coin_a:
                continue

            # we don't want to go back:
            if k.coin_b in [coin_a, j.coin_a]:
                continue
            
            coin_b = k.coin_b

            multi_hop = Route(n_hops=3, hops=[i, j, k], coin_a=coin_a, coin_b=coin_b)
            three_hops.append(multi_hop)

pprint.pprint(three_hops[0])
print(f"Number of 3 hop routes: {len(three_hops)}")

Route(n_hops=3,
      hops=[Route(n_hops=1,
                  hops=[Hop(address='0x0043fcb34e7470130fde28198571dee092c70bd7',
                            network='Mainnet',
                            coin_a=Coin(address='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                                        network='Mainnet',
                                        decimals=18),
                            coin_b=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                                        network='Mainnet',
                                        decimals=18))],
                  coin_a='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                  coin_b='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'),
            Route(n_hops=1,
                  hops=[Hop(address='0x5a6a4d54456819380173272a5e8e9b9904bdf41b',
                            network='Mainnet',
                            coin_a=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                 

In [57]:
four_hops = []
for i in one_hops:  # first hop

    coin_a = i.coin_a

    for j in one_hops:  # second hop

        # we only want connecting pools
        if not i.coin_b == j.coin_a:
            continue

        # we don't want to go back:
        if j.coin_b == coin_a:
            continue

        for k in one_hops:  # third hop

            # we only want connecting pools
            if j.coin_b != k.coin_a:
                continue

            # we don't want to go back:
            if k.coin_b in [coin_a, j.coin_a]:
                continue

            for l in one_hops:  # fourth hop

                # we only want connecting pools
                if k.coin_b != l.coin_a:
                    continue

                # we don't want to go back:
                if l.coin_b in [coin_a, j.coin_a, k.coin_a]:
                    continue
            
                coin_b = l.coin_b

                multi_hop = Route(n_hops=4, hops=[i, j, k, l], coin_a=coin_a, coin_b=coin_b)
                four_hops.append(multi_hop)

pprint.pprint(four_hops[0])
print(f"Number of 4 hop routes: {len(four_hops)}")

Route(n_hops=4,
      hops=[Route(n_hops=1,
                  hops=[Hop(address='0x0043fcb34e7470130fde28198571dee092c70bd7',
                            network='Mainnet',
                            coin_a=Coin(address='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                                        network='Mainnet',
                                        decimals=18),
                            coin_b=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                                        network='Mainnet',
                                        decimals=18))],
                  coin_a='0xa713cc74ee148414bcab46ac2c41c93d84a56b0f',
                  coin_b='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'),
            Route(n_hops=1,
                  hops=[Hop(address='0x5a6a4d54456819380173272a5e8e9b9904bdf41b',
                            network='Mainnet',
                            coin_a=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                 

Coin-route mapping


In [58]:
routes = one_hops + two_hops + three_hops + four_hops
len(routes)

14632

In [135]:
all_routes = []
for route in routes:
    all_routes.append(
        {
            "coin_a": route.coin_a,
            "coin_b": route.coin_b,
            "route": route,
            "n_hops": route.n_hops,
        }
    )

df_all_routes = pd.DataFrame(all_routes)
df_all_routes

,coin_a,coin_b,route,n_hops
0,0xa713cc74ee148414bcab46ac2c41c93d84a56b0f,0x6c3f90f043a72fa612cbac8115ee7e52bde6e490,"Route(n_hops=1, hops=[Hop(address='0x0043fcb34...",1
1,0x6c3f90f043a72fa612cbac8115ee7e52bde6e490,0xa713cc74ee148414bcab46ac2c41c93d84a56b0f,"Route(n_hops=1, hops=[Hop(address='0x0043fcb34...",1
2,0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415,0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050,"Route(n_hops=1, hops=[Hop(address='0x01fe650ef...",1
3,0x9c4a4204b79dd291d6b6571c5be8bbcd0622f050,0x15a629f0665a3eb97d7ae9a7ce7abf73aeb79415,"Route(n_hops=1, hops=[Hop(address='0x01fe650ef...",1
4,0xf006f6898a814d498927524e7a9745ec05fee18a,0x6c3f90f043a72fa612cbac8115ee7e52bde6e490,"Route(n_hops=1, hops=[Hop(address='0x021213332...",1
...,...,...,...,...
14627,0xc4ad29ba4b3c580e6d59105fff484999997675ff,0xc581b735a1688071a1746c968e0798d642ede491,"Route(n_hops=4, hops=[Route(n_hops=1, hops=[Ho...",4
14628,0xc4ad29ba4b3c580e6d59105fff484999997675ff,0xc581b735a1688071a1746c968e0798d642ede491,"Route(n_hops=4, hops=[Route(n_hops=1, hops=[Ho...",4
14629,0xc4ad29ba4b3c580e6d59105fff484999997675ff,0xdb25f211ab05b1c97d595516f45794528a807ad8,"Route(n_hops=4, hops=[Route(n_hops=1, hops=[Ho...",4
14630,0xc4ad29ba4b3c580e6d59105fff484999997675ff,0xdb25f211ab05b1c97d595516f45794528a807ad8,"Route(n_hops=4, hops=[Route(n_hops=1, hops=[Ho...",4


Get all routes for a coin. There could be redundant routes (i.e. routes with four hops vs one hops and same coin a and b).

In [157]:
coin_start = all_coins[22]
coin_end = all_coins[5]
all_routes_for_swap = df_all_routes.loc[(df_all_routes.coin_a == coin_start) & (df_all_routes.coin_b == coin_end)]
print(f"number of routes for coins: {coin_start} -> {coin_end} = {all_routes_for_swap.shape[0]}")
if not all_routes_for_swap.empty:
    print("routes: ")
    for route in all_routes_for_swap.route:
        pprint.pprint(route)

number of routes for coins: 0x513d29d2b296a51b40a35c4d63884e0ac0a8d075 -> 0x03ab458634910aad20ef5f1c8ee96f1d6ac54919 = 1
routes: 
Route(n_hops=2,
      hops=[Route(n_hops=1,
                  hops=[Hop(address='0x273afbf6e257aae160749a61d4b83e06a841c3eb',
                            network='Mainnet',
                            coin_a=Coin(address='0x513d29d2b296a51b40a35c4d63884e0ac0a8d075',
                                        network='Mainnet',
                                        decimals=18),
                            coin_b=Coin(address='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490',
                                        network='Mainnet',
                                        decimals=18))],
                  coin_a='0x513d29d2b296a51b40a35c4d63884e0ac0a8d075',
                  coin_b='0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'),
            Route(n_hops=1,
                  hops=[Hop(address='0x618788357d0ebd8a37e763adab3bc575d54c2c7d',
                            